In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from collections import defaultdict
from datetime import datetime
import h5py
import matplotlib.pyplot as plt
from neural_analysis.matIO import loadmat
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt, freqz
import sys
import time
from tqdm.auto import tqdm

sys.path.append('../..')
from ld_utils import compute_eigs, get_data_sigma_norms
from up_down import get_up_down
from utils import get_binary_stimuli, get_sample_interval, load, save, to_string

## Load Neural Data

In [4]:
# filename = '../../__data__/Mary-Anesthesia-20160809-01.mat'
filename = r'/home/adameisen/millerdata/common/datasets/anesthesia/mat/propofolPuffTone/Mary-Anesthesia-20160809-01.mat'
# filename = r'/home/adameisen/millerdata/common/datasets/anesthesia/mat/propofolPuffTone/MrJones-Anesthesia-20160121-01.mat'
# filename = r'/home/adameisen/common/datasets/anesthesia/mat/propofolWakeUp/Mary-Anesthesia-20170203-02.mat'
# filename = r'/home/adameisen/millerdata/common/datasets/anesthesia/mat/ketamine/Mary-Anesthesia-20170315-01.mat'
# filename = r'/home/adameisen/millerdata/common/datasets/anesthesia/mat/ketamine/Mary-Anesthesia-20170329-02.mat'

print("Loading data ...")
start = time.process_time()
electrode_info, lfp, lfp_schema, session_info, spike_times, trial_info, unit_info = loadmat(filename, variables=['electrodeInfo', 'lfp', 'lfpSchema', 'sessionInfo', 'spikeTimes', 'trialInfo', 'unitInfo'], verbose=False)
# electrode_info, lfp_schema, session_info, spike_times, trial_info, unit_info = loadmat(filename, variables=['electrodeInfo', 'lfpSchema', 'sessionInfo', 'spikeTimes', 'trialInfo', 'unitInfo'], verbose=False)
spike_times = spike_times[0]
dt = lfp_schema['smpInterval'][0]
T = lfp.shape[0]

f = h5py.File(filename, 'r')
# airPuff_binary, audio_binary = get_binary_stimuli(f)

print(f"Data loaded (took {time.process_time() - start:.2f} seconds)")

Loading data ...
Data loaded (took 79.50 seconds)


# Test

In [5]:
data_path = "/home/adameisen/Documents/MIT/code/ChaoticConsciousness/__data__/VAR/Mary_multipop/results_window_2.5_stride_2.5_PCA_dim_-1"

In [18]:
results = load(data_path)
params = results['params']
params['end_step'] = results['end_step']
del results['params']
del results['end_step']
del results['1step_pred']
del results['1step_pred_dist']
del results['mse']
for key in results.keys():
    if len(results[key].shape) > 1:
        results[key] = list(results[key])
results = pd.DataFrame(results)

253

In [33]:
results.window_locs

0                [0, 2500]
1             [2500, 5000]
2             [5000, 7500]
3            [7500, 10000]
4           [10000, 12500]
               ...        
2817    [7042500, 7045000]
2818    [7045000, 7047500]
2819    [7047500, 7050000]
2820    [7050000, 7052500]
2821    [7052500, 7055000]
Name: window_locs, Length: 2822, dtype: object

In [32]:
lfp[results.window_locs.iloc[i + 1][0]:results.window_locs.iloc[i + 1][0] + T_forward]

array([7052500, 7055000])

In [36]:
T_forward = 25
n_dims = results.eigs.iloc[0].shape[0] 

forward_preds = np.zeros((len(results),  T_forward, n_dims))
true_vals = np.zeros((len(results), T_forward, n_dims))
for i in range(len(results) - 1):
    row = results.loc[i]
    for t in range(T_forward):
        if t == 0:
            forward_preds[i][t] = np.concatenate([[1], lfp[results.window_locs.iloc[i + 1][0] - 1]]) @ row.A_mats
        else:
            forward_preds[i][t] = np.concatenate([[1], forward_preds[i][t-1]]) @ row.A_mats
    true_vals[i] = lfp[results.window_locs.iloc[i + 1][0]:results.window_locs.iloc[i + 1][0] + T_forward]

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 253 is different from 254)